In [1]:
import pandas as pd
import requests
import json

class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        # API Key value from link
        api_key = 'GF0IU3N4VNWYWAV5'
        
        # construct the API URL
        base_url = "https://www.alphavantage.co/query"
        function = "TIME_SERIES_INTRADAY"
        symbol = script
        interval = "1min"
        outputsize = "compact"
        api_url = f"{base_url}?function={function}&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}"
        
        # send the request and parse the response
        response = requests.get(api_url)
        if response.status_code != 200:
            raise Exception("Failed to fetch data. Check the API key and script name.")
        data = response.json()
        self.data[script] = data
    
    def convert_intraday_data(self, script):
        # get the data for the given script
        data = self.data[script]
        
        # extract the time series data
        time_series_data = data["Time Series (1min)"]
        
        # create a list of tuples containing the timestamp and data for each time point
        data_list = [(timestamp, data) for timestamp, data in time_series_data.items()]
        
        # sort the data by timestamp
        data_list.sort(key=lambda x: x[0])
        
        # extract the data into separate lists
        timestamps, data = zip(*data_list)
        opens, highs, lows, closes, volumes = [], [], [], [], []
        for d in data:
            opens.append(float(d["1. open"]))
            highs.append(float(d["2. high"]))
            lows.append(float(d["3. low"]))
            closes.append(float(d["4. close"]))
            volumes.append(int(d["5. volume"]))
        
        # create a pandas DataFrame from the lists
        df = pd.DataFrame({"timestamp": timestamps, "open": opens, "high": highs, "low": lows, "close": closes, "volume": volumes})
        
        # set the timestamp column as the index of the DataFrame
        df.set_index("timestamp", inplace=True)
        
        # convert the timestamp strings to pandas Timestamps
        df.index = pd.to_datetime(df.index)
        
        # store the DataFrame in the data dictionary
        self.data[script] = df
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value
    
    def __contains__(self, key):
        return key in self.data


In [2]:
# create an instance of the ScriptData class
script_data = ScriptData()

# fetch intraday data for the 'GOOGL' script using your Alpha Vantage API key
script_data.fetch_intraday_data('GOOGL')

# convert the fetched data into a pandas DataFrame
script_data.convert_intraday_data('GOOGL')

# access the DataFrame for the 'GOOGL' script
df = script_data['GOOGL']




In [3]:
df

,open,high,low,close,volume
timestamp,,,,,
2022-12-30 15:53:00,88.0450,88.0600,87.9144,87.9300,147762
2022-12-30 15:54:00,87.9200,87.9850,87.9200,87.9300,121464
2022-12-30 15:55:00,87.9400,88.1000,87.9200,88.0950,232475
2022-12-30 15:56:00,88.0950,88.2300,88.0800,88.1800,251081
2022-12-30 15:57:00,88.1900,88.2500,88.1750,88.2400,200370
...,...,...,...,...,...
2022-12-30 19:55:00,88.5200,88.5200,88.5200,88.5200,194
2022-12-30 19:56:00,88.5600,88.5600,88.5600,88.5600,221
2022-12-30 19:58:00,88.5699,88.5699,88.5699,88.5699,338


In [4]:

script_data.fetch_intraday_data('AAPL')

script_data.convert_intraday_data('AAPL')

df = script_data['AAPL']





In [5]:
df

,open,high,low,close,volume
timestamp,,,,,
2022-12-30 17:36:00,129.97,129.99,129.9700,129.9700,2453
2022-12-30 17:37:00,129.99,129.99,129.9900,129.9900,622
2022-12-30 17:38:00,129.97,129.99,129.9300,129.9300,1686
2022-12-30 17:39:00,129.93,130.00,129.9300,130.0000,1705
2022-12-30 17:40:00,130.01,130.03,129.9999,130.0000,2215
...,...,...,...,...,...
2022-12-30 19:56:00,129.98,129.98,129.9800,129.9800,613
2022-12-30 19:57:00,129.98,129.98,129.9700,129.9700,6624
2022-12-30 19:58:00,129.98,129.98,129.9700,129.9700,1089


In [6]:
'GOOGL' in script_data

True

In [7]:
'AAPL' in script_data

True

In [8]:
'NVDA' in script_data

False

In [9]:
def indicator1(df, timeperiod):
    # create a new column in the DataFrame with the moving average of the close price
    df["indicator"] = df["close"].rolling(timeperiod).mean()
    
    # create a new DataFrame with the timestamp and indicator columns
    result = pd.DataFrame({"timestamp": df.index, "indicator": df["indicator"]})

    return result


In [10]:
# fetch and convert intraday data for the 'GOOGL' script
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

# compute the indicator1 for the 'GOOGL' script with a timeperiod of 5
indicator1_df1 = indicator1(script_data['GOOGL'], timeperiod = 5)


In [11]:
indicator1_df1

,timestamp,indicator
timestamp,,
2022-12-30 15:53:00,2022-12-30 15:53:00,NaN
2022-12-30 15:54:00,2022-12-30 15:54:00,NaN
2022-12-30 15:55:00,2022-12-30 15:55:00,NaN
2022-12-30 15:56:00,2022-12-30 15:56:00,NaN
2022-12-30 15:57:00,2022-12-30 15:57:00,88.07500
...,...,...
2022-12-30 19:55:00,2022-12-30 19:55:00,88.47400
2022-12-30 19:56:00,2022-12-30 19:56:00,88.49600
2022-12-30 19:58:00,2022-12-30 19:58:00,88.51998


In [12]:
script_data = ScriptData()
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

indicator1_df2 = indicator1(script_data['AAPL'], timeperiod = 5)

In [13]:
indicator1_df2

,timestamp,indicator
timestamp,,
2022-12-30 17:36:00,2022-12-30 17:36:00,NaN
2022-12-30 17:37:00,2022-12-30 17:37:00,NaN
2022-12-30 17:38:00,2022-12-30 17:38:00,NaN
2022-12-30 17:39:00,2022-12-30 17:39:00,NaN
2022-12-30 17:40:00,2022-12-30 17:40:00,129.97800
...,...,...
2022-12-30 19:56:00,2022-12-30 19:56:00,129.98400
2022-12-30 19:57:00,2022-12-30 19:57:00,129.98200
2022-12-30 19:58:00,2022-12-30 19:58:00,129.97600


In [14]:
import pandas as pd

class Strategy:
    def __init__(self,script):
        self.script_data = ScriptData()
    
    def fetch_intraday_data(self, script):
        self.script_data.fetch_intraday_data(script)
        self.script_data.convert_intraday_data(script)
        self.df = self.script_data[script]
    
    def compute_indicator_data(self):
        self.indicator_df = indicator1(self.df, 5)
    
    def get_signals_data(self):
        # create a new column in the DataFrame with the moving average of the close price
        self.df["indicator"] = self.indicator_df["indicator"]
        
        # create a new column in the DataFrame with the signals
        self.df["signal"] = "NO_SIGNAL"
        self.df.loc[self.df["indicator"] > self.df["close"], "signal"] = "BUY"
        self.df.loc[self.df["indicator"] < self.df["close"], "signal"] = "SELL"
        
        # create a new DataFrame with the timestamp and signal columns
        self.initial_signals = pd.DataFrame({"timestamp": self.df.index, "signal": self.df["signal"]})
        
        # filter the signals DataFrame to include only the rows with a 'BUY' or 'SELL' signal
        self.signals = strategy.initial_signals[strategy.initial_signals["signal"].isin(["BUY", "SELL"])]
        
        


In [15]:
# create an instance of the Strategy class
strategy = Strategy('NVDA')

# fetch and compute data for the 'NVDA' script
strategy.fetch_intraday_data('NVDA')
strategy.compute_indicator_data()

# generate the signals for the 'NVDA' script
strategy.get_signals_data()

# access the signals DataFrame
signals_df = strategy.signals


In [16]:
signals_df

,timestamp,signal
timestamp,,
2022-12-30 16:13:00,2022-12-30 16:13:00,SELL
2022-12-30 16:15:00,2022-12-30 16:15:00,SELL
2022-12-30 16:16:00,2022-12-30 16:16:00,BUY
2022-12-30 16:17:00,2022-12-30 16:17:00,BUY
2022-12-30 16:18:00,2022-12-30 16:18:00,SELL
...,...,...
2022-12-30 19:47:00,2022-12-30 19:47:00,BUY
2022-12-30 19:55:00,2022-12-30 19:55:00,SELL
2022-12-30 19:56:00,2022-12-30 19:56:00,BUY
